# MDx Video Sequence (24 wells)
by Adam Zaini

This MDx sequence *only uses 4 DNA samples*. This is because the heating block can only accomodate 4 vials at a time. However, if we scale the time we could still provide a good approximate for a case of using 24 DNA samples. On top of that, further optimisation could be done.

***

## Connect robot

In [ ]:
from opentrons import robot
robot.connect()

***

## Load labware

Run to load the labware into the robot

In [ ]:
from opentrons import instruments, robot, labware

# labware and instruments involved
tiprack10 = labware.load('tiprack-10ul','4')             #10ul tip rack
tiprack300 = labware.load('opentrons-tiprack-300ul','1')  #300ul tip rack

pipette1 = instruments.P10_Single(mount='left')           #assign 10ul tip to left mount
pipette2 = instruments.P300_Single(mount='right')         #assign 300ul tip to right mount

vialrack1 = labware.load('BJS_vialrack','2')             #BJS vial rack
sample = labware.load('trough-12row','6')                 #12 row trough
xxplate = labware.load('xxpress_24well','3')              #24 well xxplate
vialrack2 = labware.load('Micronic_vialrack','7')         #Micronic vial rack
heatingblock = labware.load('heating_block','5')          #Heating block
vialrack3 = labware.load('Micronic_vialrack','8')

#### Pick up "virtual" tip to place adaptor

In [ ]:
pipette1.pick_up_tip(tiprack10.wells('H12')) # pick up tip to attach the adaptor

#### Home robot axis before starting the video

In [ ]:
robot.home()  # home axis for the start of the video

***

## Define functions

Run to ensure the kernel recognises these functions

In [ ]:
#######################################################################

# Function to fill xxplate
def fill_xxplate(well, vol, tip):
    if tip == '10':
        offset = xxplate.wells('A1').from_center(x=15.2,y=10,z=0.2)            #offset of the custom labware
        destination = (xxplate.wells(well), offset)                            #destination for the pipette (specified well)
        pipette1.move_to(destination, strategy='arc')                          #move pipette to destination
        pipette1.dispense(vol)                                                 #dispense specified volume into well
        print(well, 'has been filled by', vol, 'ul')
    elif tip == '300':
        if vol < 81:
            offset = xxplate.wells('A1').from_center(x=15.9,y=8.9,z=0.2)        #offset of the custom labware
            destination = (xxplate.wells(well), offset)                         #destination for the pipette (specified well)
            pipette2.move_to(destination, strategy='arc')                       #move pipette to destination
            pipette2.dispense(vol)                                              #dispense specified volume into well
            print(well, 'has been filled by', vol, 'ul')
        else:
            print('Well is not able to hold the specified volume.')
    else:
        print('Please select a valid tip(\'10\',\'300\')')
    return

#######################################################################

# Function to fill a vial on the BJS vial rack
def fill_BJSvial(well, vol, tip):
    if tip == '10':
        if vol < 11:                                                        
            offset = vialrack1.wells('A1').from_center(x=3.2,y=1.5,z=0)     #offset of the custom labware
            destination = (vialrack1.wells(well), offset)                   #destination for the pipette (specified well)
            pipette1.move_to(destination, strategy='arc')                   #move pipette to destination
            pipette1.dispense(vol)                                          #dispense specified volume into vial
            print(well, 'has been filled by', vol, 'ul')
        else:
            print('Tip is not able to hold the liquid volume specified.')
    elif tip == '300':
        if vol < 301:
            offset = vialrack1.wells('A1').from_center(x=3.75,y=0.8,z=0)    #offset of the custom labware
            destination = (vialrack1.wells(well), offset)                   #destination for the pipette (specified well)
            pipette2.move_to(destination, strategy='arc')                   #move pipette to destination
            pipette2.dispense(vol)                                          #dispense specified volume into vial
            print(well, 'has been filled by', vol, 'ul')
        else:
            print('Tip is not able to hold the liquid volume specified.')
    else:
        print('Please specify a valid tip (\'10\',\'300\')')
    return

#######################################################################

# Function to extract from a vial on the BJS vial rack
def extract_BJSvial(well, vol, tip):
    if tip == '10':
        if vol < 11:
            offset = vialrack1.wells('A1').from_center(x=3.2,y=1.5,z=-0.8)  #offset of the custom labware
            destination = (vialrack1.wells(well), offset)                   #destination for the pipette (specified well)
            pipette1.move_to(destination, strategy='arc')                   #move pipette to destination
            pipette1.aspirate(vol)                                          #extract specified volume from vial
            print(vol, 'ul has been extracted from', well)
        else:
            print('Volume to be extracted is greater than tip size. Please select a lower volume or a different tip')
    elif tip == '300':
        if vol < 301:
            offset = vialrack1.wells('A1').from_center(x=3.75,y=0.8,z=-0.8) #offset of the custom labware
            destination = (vialrack1.wells(well), offset)                   #destination for the pipette (specified well)
            pipette2.move_to(destination, strategy='arc')                   #move pipette to destination
            pipette2.aspirate(vol)                                          #extract specified volume from vial
            print(vol, 'ul has been extracted from', well)
        else:
            print('Volume to be extracted is greater than tip size. Please select a lower volume')
    else:
        print('Please specify a valid tip (\'10\',\'300\')')        
    return

#######################################################################

# Function to fill a vial on the Micronic vial rack
def fill_micronic(well, vol, tip):
    if tip == '10':
        offset = vialrack2.wells('A1').from_center(x=3.3,y=3.0,z=0.0)    #10ul tip
        destination = (vialrack2.wells(well), offset)
        pipette1.move_to(destination, strategy='arc')
        pipette1.dispense(vol)
        print(well, 'has been filled by', vol, 'ul')
    elif tip == '300':
        offset = vialrack2.wells('A1').from_center(x=3.8,y=2.3,z=0.0)    #300ul tip
        destination = (vialrack2.wells(well), offset)
        pipette2.move_to(destination, strategy='arc')
        pipette2.dispense(vol)
        print(well, 'has been filled by', vol, 'ul')
    return

#######################################################################

# Function to extract from a vial on the Micronic vial rack
def extract_micronic(well,vol):
    offset = vialrack3.wells('A1').from_center(x=3.8,y=2.3,z=-0.7)    #300ul tip
    destination = (vialrack3.wells(well), offset)
    pipette2.move_to(destination, strategy='arc')
    pipette2.aspirate(vol)
    print(vol, 'ul has been extracted from', well)
    return

#######################################################################

# Function to transfer vials from the micronic vial rack to the heating block
def micronic2heater(pick,dest):
    #POSITION
    offset = vialrack2.wells('A1').from_center(x=4.0,y=3.0,z=1.0)  #10ul tip
    destination = (vialrack2.wells(pick), offset)
    pipette1.move_to(destination, strategy='arc')
    #ATTACH
    offset = vialrack2.wells('A1').from_center(x=3.3,y=3.0,z=1.0)  #10ul tip
    destination = (vialrack2.wells(pick), offset)
    pipette1.move_to(destination, strategy='direct')
    pipette1.delay(seconds = 2)
    #LIFT
    offset = vialrack2.wells('A1').from_center(x=3.3,y=3.0,z=6.0)  #10ul tip
    destination = (vialrack2.wells(pick), offset)
    pipette1.move_to(destination, strategy='direct')
    pipette1.delay(seconds=1)
    #POSITION
    offset = heatingblock.wells('A1').from_center(x=9.0,y=6.6,z=2.5)
    destination = (heatingblock.wells(dest), offset)
    pipette1.move_to(destination, strategy = 'arc')
    pipette1.delay(seconds=1)
    #LOWER
    offset = heatingblock.wells('A1').from_center(x=9.0,y=6.6,z=1.6)
    destination = (heatingblock.wells(dest), offset)
    pipette1.move_to(destination, strategy = 'arc')
    #DETACH
    offset = heatingblock.wells('A1').from_center(x=9.8,y=6.6,z=1.6)
    destination = (heatingblock.wells(dest), offset)
    pipette1.move_to(destination, strategy = 'direct')
    pipette1.delay(seconds = 1)
    #RAISE
    offset = heatingblock.wells('A1').from_center(x=9.8,y=6.6,z=3.0)
    destination = (heatingblock.wells(dest), offset)
    pipette1.move_to(destination, strategy = 'direct')
    return

#######################################################################

# Function to transfer vials from the heating block to the BJS vial rack
def heater2vialrack(pick,dest):
    #POSITION
    offset = heatingblock.wells('A1').from_center(x=9.8,y=6.6,z=1.6)
    destination = (heatingblock.wells(pick), offset)
    pipette1.move_to(destination, strategy = 'arc')
    #ATTACH
    offset = heatingblock.wells('A1').from_center(x=9.0,y=6.6,z=1.6)
    destination = (heatingblock.wells(pick), offset)
    pipette1.move_to(destination, strategy = 'direct')
    pipette1.delay(seconds = 2)
    #LIFT
    offset = heatingblock.wells('A1').from_center(x=9.0,y=6.6,z=3.0)
    destination = (heatingblock.wells(pick), offset)
    pipette1.move_to(destination, strategy = 'arc')
    pipette1.delay(seconds = 1)
    #POSITION
    offset = vialrack1.wells('A1').from_center(x=2.8,y=1.5,z=2.5) #10
    destination = (vialrack1.wells(dest), offset)
    pipette1.move_to(destination, strategy='arc')
    pipette1.delay(seconds = 1)
    #LOWER
    offset = vialrack1.wells('A1').from_center(x=2.8,y=1.5,z=1.2) #10
    destination = (vialrack1.wells(dest), offset)
    pipette1.move_to(destination, strategy='arc')
    #DETACH
    offset = vialrack1.wells('A1').from_center(x=5.8,y=1.5,z=1.2) #10
    destination = (vialrack1.wells(dest), offset)
    pipette1.move_to(destination, strategy='direct')    
    #RAISE
    offset = vialrack1.wells('A1').from_center(x=5.8,y=1.5,z=3.0) #10
    destination = (vialrack1.wells(dest), offset)
    pipette1.move_to(destination, strategy='direct') 
    return

#######################################################################

# Sequence for filling xxplate with rehydration buffer
def buffer_sequence(vol):
    if 0<vol<26:                                                 #FOR THE CASE WHEN VOL IS 1 - 25
        to_extract = vol*12                                      #aspirate 12 times of the specified
        pipette2.aspirate(to_extract, sample.wells('A1'))        #aspirate
        rows = ['A','B']                                         #rows to fill first
        for r in rows:                                           #iterate over row and columns
            for c in range (1,7):
                well = r + str(c)                                #define well
                fill_xxplate(well, vol, '300')                   #fill xxplate
        rows = ['C','D']                                         #next rows to fill
        pipette2.aspirate(to_extract, sample.wells('A1'))        #aspirate
        for r in rows:                                           #iterate over rows and columns
            for c in range (1,7):
                well = r + str(c)                                #define well
                fill_xxplate(well, vol, '300')                   #fill xxplate
    elif 25<vol<51:                                              #FOR THE CASE WHEN VOL IS 26 - 50
        to_extract = vol*6
        rows = ['A','B','C','D']
        for r in rows:
            pipette2.aspirate(to_extract, sample.wells('A1'))
            for c in range (1,7):
                well = r + str(c)
                fill_xxplate(well, vol, '300')
    elif 50<vol<81:                                              #FOR THE CASE WHEN VOL IS 51 - 80
        to_extract = vol*3
        rows = ['A','B','C','D']
        for r in rows:
            pipette2.aspirate(to_extract, sample.wells('A1'))
            for c in range(1,4):
                well = r + str(c)
                fill_xxplate(well, vol, '300')
            pipette2.aspirate(to_extract, sample.wells('A1'))
            for c in range (4,7):
                well = r + str(c)
                fill_xxplate(well, vol, '300')
    else:
        print('Please specifiy a volume between 1ul and 80ul')
    return

#######################################################################

***

## Full Sequence (Single Cell)

In [ ]:
%%time

# DNA Extraction
rows = ['A','B','C','D']
int = 1
for r in rows:
    tip = 'B' + str(int)
    well_aspirate = r + '1'
    well_dispense = r + '12'
    pipette2.pick_up_tip(tiprack300.wells(tip))
    extract_micronic(well_aspirate, 40)
    fill_micronic(well_dispense, 40, '300')
    pipette2.drop_tip()
    int += 1

#Transfer vials from micronic vial rack to heating block    
micronic2heater('A12', 'B2')
micronic2heater('B12', 'A2')
micronic2heater('C12', 'B1')
micronic2heater('D12', 'A1')

#Rehydration Buffer
pipette2.pick_up_tip(tiprack300.wells('B5'))
buffer_sequence(25)
pipette2.drop_tip()

#Preload control
vol = 20
rows = ['A','B','C','D']
init = 1

pipette2.pick_up_tip(tiprack300.wells('B6'))
pipette2.aspirate(vol*8, sample.wells('A2'))
for r in rows:
    well = r + str(init)
    fill_xxplate(well, vol, '300')
    well2 = r + str(init+1)
    fill_xxplate(well2, vol, '300')
        
pipette2.drop_tip()

init = 3

pipette2.pick_up_tip(tiprack300.wells('B7'))
pipette2.aspirate(vol*8, sample.wells('A3'))
for r in rows:
    well = r + str(init)
    fill_xxplate(well, vol, '300')
    well2 = r + str(init+1)
    fill_xxplate(well2, vol, '300')
        
pipette2.drop_tip()

init = 5

pipette2.pick_up_tip(tiprack300.wells('B8'))
pipette2.aspirate(vol*8, sample.wells('A4'))
for r in rows:
    well = r + str(init)
    fill_xxplate(well, vol, '300')
    well2 = r + str(init+1)
    fill_xxplate(well2, vol, '300')
        
pipette2.drop_tip()

#Transfer vials from heating block to vial rack
heater2vialrack('B2','A12')
heater2vialrack('A2','C12')
heater2vialrack('B1','E12')
heater2vialrack('A1','G12')

#Supernatant
vol = 6
rows_xx = ['A','B','C','D']
rows = ['A','C','E','G']
init = 9
for i in range (0,4):
    tip = 'B' + str(init)
    pipette2.pick_up_tip(tiprack300.wells(tip))
    well_aspirate = rows[i] + '12'
    extract_BJSvial(well_aspirate, vol*6, '300')
    for n in range (1,7):
        well_dispense = rows_xx[i] + str(n)
        fill_xxplate(well_dispense, vol, '300')
    pipette2.drop_tip()
    init += 1

robot.home()

***

## Full Sequence (Modular Cells)

Each step is divided into separate cells, allowing the user to run each step individually

#### 1. DNA Extraction

In [ ]:
vol = 40
rows = ['A','B','C','D']
int = 1
for r in rows:
    tip = 'B' + str(int)
    well_aspirate = r + '1'
    well_dispense = r + '12'
    pipette2.pick_up_tip(tiprack300.wells(tip))
    extract_micronic(well_aspirate, vol)
    fill_micronic(well_dispense, vol, '300')
    pipette2.drop_tip()
    int += 1

#### 2. Transfering vials from Micronic vial rack to the heating block

In [ ]:
%%time
micronic2heater('A12', 'B2')
micronic2heater('B12', 'A2')
micronic2heater('C12', 'B1')
micronic2heater('D12', 'A1')

#### 3. Adding rehydration buffer to the xxplate

In [ ]:
pipette2.pick_up_tip(tiprack300.wells('B5'))
buffer_sequence(25)
pipette2.drop_tip()

#### 4. Adding preload control to the xxplate

In [ ]:
vol = 20
rows = ['A','B','C','D']
init = 1

pipette2.pick_up_tip(tiprack300.wells('B6'))
pipette2.aspirate(vol*8, sample.wells('A2'))
for r in rows:
    well = r + str(init)
    fill_xxplate(well, vol, '300')
    well2 = r + str(init+1)
    fill_xxplate(well2, vol, '300')
        
pipette2.drop_tip()

init = 3

pipette2.pick_up_tip(tiprack300.wells('B7'))
pipette2.aspirate(vol*8, sample.wells('A3'))
for r in rows:
    well = r + str(init)
    fill_xxplate(well, vol, '300')
    well2 = r + str(init+1)
    fill_xxplate(well2, vol, '300')
        
pipette2.drop_tip()

init = 5

pipette2.pick_up_tip(tiprack300.wells('B8'))
pipette2.aspirate(vol*8, sample.wells('A4'))
for r in rows:
    well = r + str(init)
    fill_xxplate(well, vol, '300')
    well2 = r + str(init+1)
    fill_xxplate(well2, vol, '300')
        
pipette2.drop_tip()

#### 5. Transferring vials from heating block to BJS vial rack

In [ ]:
%%time
heater2vialrack('B2','A12')
heater2vialrack('A2','C12')
heater2vialrack('B1','E12')
heater2vialrack('A1','G12')

pipette1.drop_tip()

#### 6. Adding supernatant to the xxplate

In [ ]:
vol = 6
rows_xx = ['A','B','C','D']
rows = ['A','C','E','G']
init = 9
for i in range (0,4):
    tip = 'B' + str(init)
    pipette2.pick_up_tip(tiprack300.wells(tip))
    well_aspirate = rows[i] + '12'
    extract_BJSvial(well_aspirate, vol*6, '300')
    for n in range (1,7):
        well_dispense = rows_xx[i] + str(n)
        fill_xxplate(well_dispense, vol, '300')
    pipette2.drop_tip()
    init += 1

#### 7. Clear commands and reset robot

In [ ]:
robot.clear_commands()
robot.reset()